## 9-4 衣料品画像の識別（CNN）

### ライブラリのインポート

In [ ]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout 
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline

- tensorflow.keras.layers
  - Conv2D：畳み込み層を定義するための関数
  - MaxPooling2D：プーリング層を定義するための関数

### データの読み込み

In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

- (x_train, y_train), (x_test, y_test) = fashion.mnist.load_data()
  - Fashion MNISTデータを読み込んで、訓練データとテストデータへ格納する。
  - 初回実行時にインターネット経由でデータがダウンロードされる。
- class_names = [ ･･･ ]
  - クラス毎の名称（衣料品の種類）を設定したリスト

### 先頭の画像25枚の表示

In [ ]:
fig = plt.figure(figsize=(8,8))
for i in range(25):
    ax = fig.add_subplot(5, 5, i+1, xticks=[], yticks=[],
                        xlabel=class_names[y_train[i]])
    ax.imshow(x_train[i].reshape(28, 28), cmap='gray_r')

- ax = fig.add_subplot(5, 5, i+1, xticks=[], yticks=[], 
  - xlabel=class_names[y_train[i]])
  - サブプロットを追加して、クラス名をラベルで表示する。
- ax.imshow(x_train[i].reshape((28, 28)), cmap='gray_r')
  - 画像を描画する。cmap='gray_r' はグレースケールで白黒反転の意味。

### 訓練／テストデータの作成

In [ ]:
# データの正規化（最小値 0、最大値 1）
x_trainc = x_train.reshape(x_train.shape[0], 28, 28, 1) / 255.0
x_testc = x_test.reshape(x_test.shape[0], 28, 28, 1) / 255.0
# ラベルデータのOne-Hotベクトル化
y_trainc = to_categorical(y_train, 10)
y_testc = to_categorical(y_test, 10)

- x_trainc, x_testc
  - 元データの値0〜255を0〜1に変換（Min-Max正規化）したもの。
- x_*.reshape(x_*.shape[0], 28, 28, 1)
  - 機械学習モデルの入力形式に合わせるため次元数を変換する。
- y_trainc, y_testc
  - 正解ラベルを2値化（One-hotベクトル化）したもの。

### モデルの構築

In [ ]:
model = Sequential()
# 畳み込み層
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
# プーリング層
model.add(MaxPooling2D(pool_size=(2, 2)))
# ドロップアウト
model.add(Dropout(0.25))
# 全結合層・ドロップアウト
model.add(Flatten())
model.add(Dense(128, activation='relu'))
# ドロップアウト
model.add(Dropout(0.5))
# 出力層
model.add(Dense(10, activation='softmax'))

- Sequential APIでモデルを定義する。

### 学習の実行

In [ ]:
%%time
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(x_trainc, y_trainc,
              batch_size=64,
              epochs=10,
              validation_split=0.2)

- %%time
  - 実行時間の測定（参考：CPU単独 約10分、GPU使用 約1分）
- model.complie：学習のためのモデルの設定
  - 損失関数: categorical_crossentropy （交差エントロピー）
  - 最適化アルゴリズム: Adam
  - 評価関数: accuracy
- model.fit：機械学習の実行
  - バッチサイズ: 64
  - エポック数: 10
  - 検証用データの割合: 0.2 （80%のデータで学習）

### モデルの評価

In [ ]:
# 分類予測の取得
pred = np.argmax(model.predict(x_testc), axis=-1)
# 正解率と混同行列の出力
print('Test Accuracy:\n', accuracy_score(y_test, pred)) 
print('Confusion Matrix:\n', confusion_matrix(y_test, pred))

- 評価用データを使って、学習済みモデルの分類性能を評価する。